In [1]:
from mode import *

In [2]:
keys = [np.random.randint(0, 2**64, dtype=np.uint64) for _ in range(2)]
print("Keys:")
print(f"{keys[0]:x}")
print(f"{keys[1]:x}")


Keys:
ddd52ed8613d33e8
6b6e5c72e5f95bc4


In [3]:
from bloc import *

message = np.random.randint(0, 2**64, dtype=np.uint64)

print("Message:")
print(f"{message:x}")

cipher, num_blocks = block_encrypt(keys[0], message)
print("Cipher:")
print(f"{cipher:x}")


plaintext = block_decrypt(keys[0], cipher, num_blocks=num_blocks)
print("Plaintext:")
print(f"{plaintext:x}")

print("Verification:")
print(message == plaintext)



Message:
546be33d0ed3c26b
Cipher:
6634e325a4ca6f57
Plaintext:
546be33d0ed3c26b
Verification:
True


/Users/neosapien/Development/exam-sg8/sbox.py:24: RuntimeWarning: overflow encountered in scalar multiply
  x *= b
/Users/neosapien/Development/exam-sg8/sbox.py:27: RuntimeWarning: overflow encountered in scalar multiply
  x *= a
/Users/neosapien/Development/exam-sg8/sbox.py:33: RuntimeWarning: overflow encountered in scalar multiply
  x *= a_inv
/Users/neosapien/Development/exam-sg8/sbox.py:36: RuntimeWarning: overflow encountered in scalar multiply
  x *= b_inv


In [5]:
message = int(''.join([str(np.random.randint(0, 2**64, dtype=np.uint64)) for _ in range(16)]))
print("Message:")
print(f"{message:x}")

aux, cipher, tag = encrypt_and_authenticate(keys, message)
print("Cipher:")
print(f"{cipher:x}")
print("Tag:")
print(f"{tag:x}")

#cipher += 1

verified, decrypted = decrypt_and_verify(keys, cipher, tag, aux)
print("Verified:")
print(verified)
if verified:
    print("Decrypted:")
    print(f"{decrypted:x}")


Message:
11170fe789106b66f738bb490ba14c6dd805cb5ab3e5f162d4915988550309a4b714a00c2c4d395d5a18b01f6f748db868fa260fcfeffdc763f33f62acc9b88aed808032c84dd091db2003f295c0327d6d51a6d0184458c385bef5a01f48b7019e2a4d3173bd8ce96de278019f960f519e0baa435eb52f980ad426f216079
Cipher:
4576b63f49b619b97cdb48ce08430a9bcb7a6fd3cd0016c3edfd4cfdec87767970bdbda154157a8055b449c633f32b4066516bc9738597e2cbeb58811c4776f255794bb9b0f53518d11c15fbd0a72c5e6b6b1a67240f99bac8b92104b0ed10b7b16cc5e4b951058fd6373c13d9ea1d49f9c35e7efc9fa0d65f030ca975175272
Tag:
210ffbf82a62b9299e083c6c3be9e80e
Verified:
True
Decrypted:
11170fe789106b66f738bb490ba14c6dd805cb5ab3e5f162d4915988550309a4b714a00c2c4d395d5a18b01f6f748db868fa260fcfeffdc763f33f62acc9b88aed808032c84dd091db2003f295c0327d6d51a6d0184458c385bef5a01f48b7019e2a4d3173bd8ce96de278019f960f519e0baa435eb52f980ad426f216079


In [12]:
import random
from hash16 import *  

def difference_hash_in_and_out(start, size_m, steps=1000):
    current = start
    print(f"Départ : {hex(current)}")
    previous = start
    previous_hash = hash(start)
    diff = 0
    s = 0
    for i in range(steps):
        # Mar​che aléatoire : flip 1 bit aléatoire
        bit_to_flip = 1 << random.randint(0, size_m-1)
        current ^= bit_to_flip
        h = hash(current)
        diff = h ^ previous_hash
        #print(f"Étape {i+1:4d} : message_diff = {bin(previous ^ current)} | diff_hash = {bin(diff)}")
        s += bin(diff).count('1')
        previous = current
        previous_hash = h
    return s/steps

size_m = 256
start_msg = random.getrandbits(size_m)
difference_hash_in_and_out(start_msg, size_m, steps=1000)

Départ : 0xb8eb3b732bb784bf1101893556982bb433da7e92608fc6fcee78c24cc30c261e


5.98